In [1]:
import numpy as np
from bigfile import BigFile
import struct
import glob
import os, sys

In [6]:
class BHType:
    def __init__(self, name_sel=None):
        self.name_all = ('size','BHID','BHMass','Mdot','Density','timebin','Encounter','MinPos',\
             'MinPot','Entropy','GasVel','acMom','acMass','acBHMass',\
             'Fdbk','SPHID','SwallowID','CountProgs','Swallowed',\
             'BHpos','srDensity','srParticles','srVel','srDisp',\
             'DFAccel','DragAccel','GravAccel','BHvel','Mtrack','Mdyn',\
             'KineticFdbkEnergy','NumDM','V1sumDM','V2sumDM','MgasEnc','KEflag','z','size2')
        self.dtype_all = ('i','q','d','d','d','i','i','3d',\
            'd','d','3d','3d','d','d',\
            'd','q','q','i','i',\
            '3d','d','d','3d','d',\
            '3d','3d','3d','3d','d','d',\
            'd','d','3d','d','d','i','d','i')

        if name_sel is None:
            name_sel = self.name_all
        self.name_sel = name_sel
        self.name2type = {name: dtype for name, dtype in zip(self.name_all, self.dtype_all)}


    @property
    def TypeAll(self):
        np_type = np.dtype({'names':self.name_all, 'formats':self.dtype_all})
        return np_type
    @property
    def TypeSel(self):
        name_sel = [name for name in self.name_all if name in name_sel]
        dtype_sel = [dtype for name, dtype in zip(self.name_all, self.dtype_all) if name in name_sel]
        np_type = np.dtype({'names':name_sel, 'formats':dtype_sel})
        return np_type

In [111]:
np.seterr(all='raise')
def read_binary_file(file):
    dt = BHTYPE.TypeAll
    dtype = BHTYPE.dtype_all
    #-----------------------------------------------------------
    struct_fmt = ''.join(list(dtype))
    struct_len = struct.calcsize(struct_fmt)
    data = np.zeros(1, dtype=BHTYPE.TypeAll)
    nsingle = data.nbytes
    
    print(nsingle)
    print(struct_len)
    
    results = np.fromfile(file, dtype=dt, count=-1, offset=0)
    hastrouble = 0
    
    mask =  (results["size"] != nsingle - 8) | (results["size2"] != nsingle - 8)
    mask = mask.nonzero()[0]
    if len(mask > 0):
        hastrouble = 1
        istart = mask[0]
        ntrouble = sum(mask)
        print(f"starting of trouble {istart}, total troubles {ntrouble}")
        print(results[istart - 1])
        # print("================================")
        # for dd in BHTYPE.name_all:
        #     print(dd, results[istart][dd])

        offset = istart * nsingle
        print("=============start of bad data=============")
        trouble_data = np.fromfile(file, dtype=dt, count=1, offset=offset)
        for dd in ["size", "size2"]:
            print(dd, trouble_data[dd][0])
            
        # search for restart point
        for newoff in range(offset + 4, offset + nsingle):
            num = np.fromfile(file, dtype='i', count=1, offset=newoff)
            if num == nsingle - 8:
                print(f"resetting offset from {offset} to {newoff}")
                break
        print("=============good data=============")
        next_data = np.fromfile(file, dtype=dt, count=1, offset=newoff)
        for dd in ["size", "size2"]:
            print(dd, next_data[dd][0])

    
    return results, hastrouble

In [112]:


SEL_COL=['acBHMass', 'BHMass', 'Density', 'KEflag',  'Mdyn', 'NumDM', 'V1sumDM',
'acMass', 'BHpos', 'Encounter', 'KineticFdbkEnergy','MgasEnc', 'Swallowed', 'V2sumDM',
'BHID', 'BHvel', 'Entropy', 'DFAccel','DragAccel','GravAccel', 'Mdot', 'Mtrack', 'SwallowID',  'z']

BHTYPE = BHType(SEL_COL)

fflist = sorted(glob.glob("/home1/08942/nianyic/ASTRID2_PIG/BlackholeDetails-R599/*"))

trouble_counter = 0
for i in range(10):
    print("----- %d --------"%i)
    data, trouble = read_binary_file(fflist[i])
    trouble_counter += trouble
print(trouble_counter)

----- 0 --------
436
444
starting of trouble 66024, total troubles 29576354844
(428, 167252641036, 0.00017778, 3.09355878e-05, 5.40947514e-05, 35, 0, [-1.06304220e+01,  3.41840975e+03,  2.51907262e+04], -3034722.48403937, 1003965.69408295, [-226.38115628,   20.0318527 , -101.82594471], [0., 0., 0.], 0., 0., 5.40947514e-05, 0, -1, 0, 0, [-6.75028650e+00,  3.41679462e+03,  2.51923161e+04], 0.00095709, 0., [-232.06602987,   24.55486886,  -96.12712853], 254.42304363, [ 0.00714018,  0.02368753, -0.08739135], [ 0.10733834, -0.05835113, -0.18150061], [ 53.12678756,  41.77323721, -63.69144304], [-232.43706587,   23.32395842,  -91.58588212], 0.00025548, 0.0009958, 0., 0., [0., 0., 0.], 29.20481091, 0.05082452, 0, 0.57054673, 428)
=============start of bad data=============
size 428
size2 1063055456
resetting offset from 28786464 to 28786688
=============good data=============
size 428
size2 428
----- 1 --------
436
444
starting of trouble 80755, total troubles 43861552893
(428, 173001158620, 0.